#### S0 - 정상 업무 대화 데이터셋 생성기

이 스크립트는 **OpenAI API**를 사용해 **기업 내부 정상 업무 대화 데이터셋**을 생성합니다.  
총 **559개의 시나리오**를 기반으로, 각 시나리오당 1~2개의 대화를 만들어 **약 1,000개 데이터**를 생성합니다.

---

✅ 주요 기능
- **자연스러운 한국인 이름 자동 생성**
  - `KoreanNameGenerator`를 이용해 중복 없는 자연스러운 이름을 자동 생성
- **100개 단위로 분할 저장**
  - 1,000개 데이터를 10개 JSON 파일로 나누어 저장
- **API 실 호출 기반 생성**
  - 실제 OpenAI API 호출로 데이터 생성
  - 총 토큰 사용량 출력

---

⚙️ 실행 순서
1. API 키 입력 → OpenAI 클라이언트 초기화  
2. 시나리오 목록 로드 (559개)  
3. 이름 자동 생성 (중복 방지)  
4. 대화 생성 (시나리오별 1~2개)  
5. 100개 단위로 JSON 파일 저장  
6. 총 토큰 사용량 출력  

In [1]:
import os
import json
import time
import random
from typing import Dict, List, Optional, Set
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
from datetime import datetime

1. 설정 및 상수

In [ ]:
# API 설정
OPENAI_MODEL = 'gpt-4o'
API_DELAY_SECONDS = 1.0

# 생성 설정
TOTAL_TARGET_SAMPLES = 1000  # 총 생성할 데이터 수
BATCH_SIZE = 100  # 배치당 데이터 수
NUM_BATCHES = 10  # 총 배치 수

# 출력 설정
OUTPUT_DIR = "s0_generated_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

2. 한국인 이름 생성기

In [4]:
import random
from typing import List, Set

class KoreanNameGenerator:
    """중복 없는 자연스러운 한국인 이름 생성기 (성과 이름 첫 글자 중복 방지)"""

    # 한국 성씨
    SURNAMES = [
        "김", "이", "박", "최", "정", "강", "조", "윤", "장", "임",
        "한", "오", "서", "신", "권", "황", "안", "송", "전", "홍",
        "유", "고", "문", "양", "손", "배", "백", "허", "남", "심",
        "노", "하", "곽", "성", "차", "주", "우", "구", "민", "류"
    ]

    # 이름 200개
    GIVEN_NAMES = [
        "민석", "서준", "도윤", "하준", "주원", "시우", "지호", "지후", "유준", "현우",
        "하윤", "준우", "예준", "건우", "승우", "민재", "연우", "준호", "시윤", "현준",
        "예진", "하연", "유주", "소윤", "유진", "민서", "지유", "세은", "주하", "다현",
        "주호", "지훈", "준혁", "하민", "도훈", "지헌", "하성", "유찬", "민호", "승호",
        "서연", "하은", "지우", "서윤", "지민", "수아", "하린", "예린", "채원", "유나",
        "지원", "지윤", "다은", "수빈", "아린", "하율", "예은", "서하", "우빈", "성현",
        "도현", "시현", "태현", "민성", "승민", "지안", "현민", "나은", "가은", "하영",
        "시연", "서우", "예원", "민하", "채윤", "예서", "승현", "진우", "은우", "태윤",
        "현서", "우진", "재윤", "성민", "재원", "지환", "지현", "소율", "하늘", "은서",
        "서희", "예나", "수연", "서린", "유리", "라윤", "지한", "도연", "태민", "하진",
        "윤호", "하람", "지혁", "민결", "은찬", "유담", "준영", "재민", "하온", "윤서",
        "서진", "은호", "민우", "재현", "예담", "지완", "유성", "세윤", "나윤", "다온",
        "서빈", "도하", "현진", "윤재", "가율", "하빈", "은재", "민규", "유건", "시훈",
        "태율", "민솔", "도겸", "유하", "하경", "주은", "태린", "지율", "현이", "민채",
        "서현", "라희", "하엘", "지영", "은별", "도아", "예슬", "나리", "수현", "가빈",
        "세영", "연진", "나율", "지효", "라온", "민교", "서유", "예솔", "주리", "윤아",
        "은유", "유민", "도유", "하음", "시온", "유현", "예아", "연서", "하예", "재하",
        "도율", "하랑", "서담", "지유", "라미", "나현", "가현", "채린", "예온", "시안",
        "도진", "지혁", "은지", "민호", "유찬", "하율", "나윤", "가온", "도은", "예린"
    ]

    def __init__(self):
        self.used_names: Set[str] = set()
        self.generation_count = 0

    def generate_name(self, max_attempts: int = 100) -> str:
        """중복 없는 한국인 이름 생성 (성과 이름 첫 글자 중복 방지)"""
        for _ in range(max_attempts):
            surname = random.choice(self.SURNAMES)

            # 이름 후보 중에서 성의 첫 글자와 다르게 시작하는 이름만 필터링
            valid_given_names = [name for name in self.GIVEN_NAMES if not name.startswith(surname)]
            if not valid_given_names:
                valid_given_names = self.GIVEN_NAMES  # 혹시 전부 제외될 경우 대비

            given_name = random.choice(valid_given_names)
            full_name = surname + given_name

            if full_name not in self.used_names:
                self.used_names.add(full_name)
                self.generation_count += 1
                return full_name

        # 중복 시 숫자 추가 (fallback)
        surname = random.choice(self.SURNAMES)
        given_name = random.choice(self.GIVEN_NAMES)
        full_name = f"{surname}{given_name}{self.generation_count}"
        self.used_names.add(full_name)
        self.generation_count += 1
        return full_name

    def generate_names(self, count: int) -> List[str]:
        """여러 개의 중복 없는 이름 생성"""
        return [self.generate_name() for _ in range(count)]

    def reset(self):
        """사용된 이름 초기화"""
        self.used_names.clear()
        self.generation_count = 0


# 전역 이름 생성기 초기화
name_generator = KoreanNameGenerator()

print("✅ 한국인 이름 생성기 초기화 완료")
print("\n테스트 생성 이름 10개:")
for i, name in enumerate(name_generator.generate_names(10), 1):
    print(f"  {i}. {name}")

✅ 한국인 이름 생성기 초기화 완료

테스트 생성 이름 10개:
  1. 남유찬
  2. 곽예솔
  3. 류하은
  4. 차유찬
  5. 고예슬
  6. 안지호
  7. 주서하
  8. 성태윤
  9. 송태민
  10. 장유현


3. 토큰 추적기

In [5]:
class TokenTracker:
    """토큰 사용량 추적 클래스"""
    
    def __init__(self):
        self.total_prompt_tokens = 0
        self.total_completion_tokens = 0
        self.total_tokens = 0
        self.request_count = 0
        self.batch_stats = []  # 배치별 통계
    
    def add_usage(self, usage):
        """API 응답에서 토큰 사용량 추가"""
        self.total_prompt_tokens += usage.prompt_tokens
        self.total_completion_tokens += usage.completion_tokens
        self.total_tokens += usage.total_tokens
        self.request_count += 1
    
    def add_batch_stats(self, batch_num: int, samples: int, tokens: int):
        """배치 통계 추가"""
        self.batch_stats.append({
            'batch': batch_num,
            'samples': samples,
            'tokens': tokens
        })
    
    def get_summary(self):
        """사용량 요약 반환"""
        return {
            'total_requests': self.request_count,
            'prompt_tokens': self.total_prompt_tokens,
            'completion_tokens': self.total_completion_tokens,
            'total_tokens': self.total_tokens,
            'avg_tokens_per_request': round(self.total_tokens / self.request_count, 2) if self.request_count > 0 else 0
        }
    
    def print_summary(self):
        """사용량 요약 출력"""
        summary = self.get_summary()
        print("\n" + "="*80)
        print("📊 토큰 사용량 요약")
        print("="*80)
        print(f"총 API 요청 수: {summary['total_requests']:,}회")
        print(f"\n토큰 사용량:")
        print(f"  • 입력 토큰:    {summary['prompt_tokens']:,}")
        print(f"  • 출력 토큰:    {summary['completion_tokens']:,}")
        print(f"  • 총 토큰:      {summary['total_tokens']:,}")
        print(f"  • 평균/요청:    {summary['avg_tokens_per_request']:,.2f}")
        
        if self.batch_stats:
            print(f"\n배치별 통계:")
            for stat in self.batch_stats:
                print(f"  배치 {stat['batch']:2d}: {stat['samples']:3d}개 샘플, {stat['tokens']:,} 토큰")
        
        print("="*80)


# 전역 토큰 추적기
token_tracker = TokenTracker()
print("✅ 토큰 추적기 초기화 완료")

✅ 토큰 추적기 초기화 완료


4. 시나리오 로드

In [6]:
# 시나리오 로드
with open('S0_scenarios.json', 'r', encoding='utf-8') as f:
    ALL_SCENARIOS = json.load(f)

print(f"✅ 총 {len(ALL_SCENARIOS)}개의 시나리오를 로드했습니다.")
print(f"\n샘플 시나리오 (처음 3개):")
for i, s in enumerate(ALL_SCENARIOS[:3]):
    print(f"  {i+1}. {s['type']} - {s['participants']} - {s['context']}")

✅ 총 559개의 시나리오를 로드했습니다.

샘플 시나리오 (처음 3개):
  1. 서버 상태 보고 - 운영팀 내부 - 정기 서버 점검 결과 공유
  2. DB 백업 협의 - DB팀과 보안팀 - 백업 일정 및 절차 논의
  3. 프로젝트 진행 상황 공유 - 개발팀 내부 - 주간 프로젝트 미팅


5. 시스템 프롬프트

In [8]:
SYSTEM_INSTRUCTION = """당신은 기업 내부 정상 업무 대화 데이터 생성 전문가입니다.

**목적**: 개인정보 보호 업무를 수행하는 직원들의 일상적인 업무 대화를 생성합니다.

**출력 형식**: 반드시 유효한 JSON만 출력하세요.
```json
{
  "dialogue": "발화1\\n발화2\\n발화3...",
  "label": 0,
  "meta": {
    "conversation_type": "work_normal",
    "participants_role": "보안팀 내부",
    "context": "서버 점검 결과 공유",
    "sensitive_info_included": true,
    "fake_values": {"ip": "10 23 45 67"},
    "tone": "professional",
    "time_context": "업무시간",
    "call_environment": "(아래 옵션 중 하나를 랜덤하게 선택)"
  }
}
```

**call_environment 옵션 (균등한 비율로 사용되도록 다양한 선택을 할 것)**:
1. "시끄러운 실내" - 카페, 식당, 쇼핑몰, 헬스장
2. "조용한 실내" - 집, 사무실, 개인 공간
3. "실외" - 거리, 공원, 주차장
4. "이동 중" - 걷는 중, 버스/지하철 안, 차 안
5. "조용한 공공장소" - 도서관, 병원 대기실, 은행
- call_environment는 위 다섯 가지 중 하나를 무작위(random)로 선택할 것.
- **균등 확률(각 20%)**로 선택되도록 가정하며, 특정 항목이 반복적으로 선택되지 않도록 주의한다.
- 여러 샘플을 생성할 때는 내부적으로 주사위를 굴리듯 난수를 기반으로 분포를 제어한다고 생각하고,
한 세션 내에서는 동일한 환경이 연속적으로 나타나지 않도록 한다.
- 무작위성은 문맥보다 우선하며, 각 환경의 표현이 자연스럽게 드러나도록 배경 묘사를 조정한다.

**핵심 규칙**:
1. **label = 0 (정상 대화)**
2. 업무상 필요에 의해 민감정보가 자연스럽게 언급될 수 있음
3. 모든 민감정보는 허구값 (IP: 10.x.x.x, 192.168.x.x만)
4. STT 형식: 소문자, 특수문자 없음, 발화는 \\n으로 구분

5. **리얼한 업무 대화 표현**:
   - 간투사: "음", "어", "그", "아" (적당히)
   - 업무 용어: "확인 부탁드립니다", "검토 후 회신드리겠습니다"
   - 정중한 어투 유지

6. **통화 환경 디테일 (자연스럽게 반영)**:
   - 배경 상황: "지금 회의 중이라 잠깐만요", "아 지금 좀 바쁘긴 한데"
   - 통화 품질: "어 잠깐 소리가 잘 안 들리네요", "네 다시 한 번 말씀해주세요"
   - 멀티태스킹: "음 자료 찾아보면서 말씀드릴게요", "잠시만요 확인 중입니다"

7. **감정/상태 표현**:
   - 바쁨: "음 급하긴 한데 괜찮으시죠", "빠르게 처리 부탁드립니다"
   - 피로: "오늘 일이 좀 많아서", "음 지금 좀 정신없긴 하네요"
   - 친근함: "수고하셨어요", "네 알겠습니다 감사합니다"

8. **실제 업무 프로세스 반영**:
   - 신원 확인: "혹시 성함이 어떻게 되세요", "음 부서가 어디신가요"
   - 정보 재확인: "제가 다시 한번 확인할게요", "음 그럼 정리하면"
   - 후속 조치: "이메일로도 보내드릴게요", "티켓 남겨두겠습니다"
   - 대기/보류: "잠시만 기다려주세요", "음 확인하고 다시 연락드리겠습니다"

9. **업무상 정보 공유 원칙 (개인정보보호법 준수)**:
   기술 용어는 자연스럽게 사용하되, 정보 공유 원칙을 반드시 준수한다
   **A. 정보 요청 최소화**:
   - 업무 목적에 **반드시 필요한** 정보만 요청
   - 예시:
     * 서버 점검: 서버명, IP 대역만 (접속 계정 불필요)
     * 로그 확인: 로그 경로만 (전체 시스템 정보 불필요)
     * 백업 협의: 백업 일정, 경로만 (관리자 계정 불필요)

   **B. 요청 시 반드시 목적과 범위 명시**:
   - "장애 대응을 위해 해당 서버 IP 대역만 확인 부탁드립니다"
   - "감사 대응용으로 최근 일주일 로그 경로만 알려주시겠어요"
   - "백업 검증 목적으로 파일명만 확인할게요"

   **C. 민감 정보는 대체 수단 사용**:
   - 비밀번호/키: "이메일로 암호화해서 보내주세요"
   - 접근 권한: "티켓으로 요청 올려주시면 검토하겠습니다"
   - 상세 설정: "문서 링크 공유 부탁드립니다"

   **D. 과도한 정보 제공 시 제재**:
   - "음 그건 전화로 말씀하시면 안 되고요"
   - "보안상 그 정보는 이메일로 주시는 게 좋겠습니다"
   - "접근 권한 정보는 별도 승인 절차 필요합니다"

   **E. 절대 전화로 공유하지 않는 정보**:
   - 계정 비밀번호
   - API 키 전체
   - DB 접속 정보 전체
   - 관리자 권한 정보
   - 암호화 키

   **F. 정보 공유 전 확인 절차**:
   - "먼저 티켓 번호 말씀해주시겠어요"
   - "음 부서 어디신지 확인 좀 할게요"
   - "이메일로 요청 주시면 검토 후 회신드리겠습니다"

10. 대화 길이: 8-15턴
11. 양측 모두 정당한 업무 목적으로 대화
12. 의심/경계 없이 협조적인 분위기
13. 플레이스홀더 금지, 실존 기관명 금지
14. 화자 표시 없이 발화만 출력
"""

print("✅ 시스템 프롬프트 로드 완료")

✅ 시스템 프롬프트 로드 완료


6. OpenAI 클라이언트 초기화

In [9]:
def initialize_client() -> Optional[OpenAI]:
    """OpenAI 클라이언트 초기화 (input으로 키 입력)"""
    
    # 환경변수에 없으면 직접 입력 요청
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        api_key = input("🔑 OpenAI API 키를 입력하세요: ").strip()

    if not api_key:
        print("❌ API 키가 입력되지 않았습니다.")
        return None

    try:
        client = OpenAI(api_key=api_key)
        print("✅ OpenAI 클라이언트 초기화 완료")
        return client
    except Exception as e:
        print(f"❌ 클라이언트 초기화 실패: {e}")
        return None


# 실행 시 입력 요청됨
client = initialize_client()

✅ OpenAI 클라이언트 초기화 완료


7. 대화 생성 함수

In [10]:
def generate_single_conversation(
    client: OpenAI,
    scenario: Dict,
    participant_names: List[str]
) -> Optional[Dict]:
    """단일 대화 생성"""
    
    # 통화 환경 선택
    environments = [
        "조용한 사무실",
        "회의실",
        "재택근무 중",
        "외근 중",
        "이동 중"
    ]
    selected_environment = random.choice(environments)
    
    # 시간 컨텍스트 선택
    time_contexts = [
        "업무시간",
        "점심시간",
        "오전 회의 전",
        "오후 업무 중",
        "퇴근 직전"
    ]
    selected_time = random.choice(time_contexts)
    
    user_prompt = f"""
시나리오: {scenario['type']}
참여자: {scenario['participants']}
상황: {scenario['context']}
정보 유형: {scenario['info_type']}

대화 참여자 이름: {', '.join(participant_names)}

**중요: 통화 환경은 "{selected_environment}"로 설정할 것**
**중요: 시간 컨텍스트는 "{selected_time}"로 설정할 것**

위 시나리오에 맞는 자연스러운 업무 전화 대화를 생성하세요.
대화에 이름을 직접 언급하지 말고, meta 정보의 participants에만 포함하세요.
"""
    
    try:
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_INSTRUCTION},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.9,
            response_format={"type": "json_object"}
        )
        
        # 토큰 사용량 추적
        token_tracker.add_usage(response.usage)
        
        # 응답 파싱
        content = response.choices[0].message.content
        data = json.loads(content)
        
        # 메타 정보에 참여자 추가
        if 'meta' not in data:
            data['meta'] = {}
        data['meta']['participants'] = participant_names
        
        # 시나리오 정보 추가
        data['scenario_type'] = scenario['type']
        data['scenario_participants'] = scenario['participants']
        data['scenario_context'] = scenario['context']
        
        return data
        
    except json.JSONDecodeError as e:
        print(f"  ⚠️  JSON 파싱 실패: {e}")
        return None
    except Exception as e:
        print(f"  ⚠️  생성 실패: {e}")
        return None


print("✅ 대화 생성 함수 정의 완료")

✅ 대화 생성 함수 정의 완료


8. 배치 생성 함수

In [11]:
def generate_batch(
    client: OpenAI,
    batch_num: int,
    scenarios: List[Dict],
    target_samples: int,
    delay: float = 1.0
) -> pd.DataFrame:
    """
    배치 단위로 대화 데이터 생성
    
    Args:
        client: OpenAI 클라이언트
        batch_num: 배치 번호
        scenarios: 사용할 시나리오 리스트
        target_samples: 목표 샘플 수
        delay: API 호출 간 딜레이
    """
    print(f"\n{'='*80}")
    print(f"📦 배치 {batch_num} 생성 시작 (시나리오 {len(scenarios)}개 → 목표 {target_samples}개)")
    print(f"{'='*80}\n")
    
    conversations = []
    batch_start_tokens = token_tracker.total_tokens
    start_time = time.time()  # 시작 시간 기록
    
    # 각 시나리오당 생성할 샘플 수 계산
    samples_per_scenario = target_samples // len(scenarios)
    remaining_samples = target_samples % len(scenarios)
    
    for idx, scenario in enumerate(scenarios, 1):
        # 이 시나리오에서 생성할 샘플 수
        num_samples = samples_per_scenario
        if idx <= remaining_samples:
            num_samples += 1
        
        if num_samples == 0:
            continue
        
        scenario_success = 0
        scenario_fail = 0
        scenario_names = []  # 생성된 이름들 저장
        
        for sample_idx in range(num_samples):
            # 참여자 이름 생성 (2명)
            participant_names = name_generator.generate_names(2)
            
            # 대화 생성
            conversation = generate_single_conversation(
                client=client,
                scenario=scenario,
                participant_names=participant_names
            )
            
            if conversation:
                conversations.append(conversation)
                scenario_success += 1
                scenario_names.append(f"({', '.join(participant_names)})")
            else:
                scenario_fail += 1
            
            # API 호출 간 딜레이
            time.sleep(delay)
        
        # 시나리오별 결과를 한 줄로 출력 (이름 포함)
        status = "✅" if scenario_fail == 0 else "⚠️"
        names_str = " ".join(scenario_names) if scenario_names else ""
        print(f"{status} [{idx}/{len(scenarios)}] {scenario['type']}: {scenario_success}개 생성 {names_str}" + 
              (f" ({scenario_fail}개 실패)" if scenario_fail > 0 else ""))
        
        # 진행바 + 예상 시간 표시
        if idx % 5 == 0 or idx == len(scenarios):
            progress = idx / len(scenarios)
            bar_length = 40
            filled = int(bar_length * progress)
            bar = '█' * filled + '░' * (bar_length - filled)
            
            # 경과 시간 및 예상 남은 시간 계산
            elapsed_time = time.time() - start_time
            avg_time_per_scenario = elapsed_time / idx
            remaining_scenarios = len(scenarios) - idx
            estimated_remaining = avg_time_per_scenario * remaining_scenarios
            
            # 시간 포맷팅
            elapsed_str = f"{int(elapsed_time//60)}:{int(elapsed_time%60):02d}"
            if estimated_remaining > 0:
                remaining_str = f"{int(estimated_remaining//60)}:{int(estimated_remaining%60):02d}"
                time_info = f"[{elapsed_str}<{remaining_str}, {avg_time_per_scenario:.2f}s/it]"
            else:
                time_info = f"[{elapsed_str}, {avg_time_per_scenario:.2f}s/it]"
            
            print(f"📊 배치 {batch_num}: [{bar}] {progress*100:.1f}% ({idx}/{len(scenarios)}) {time_info} | 생성: {len(conversations)}개\n")
    
    # 배치 통계
    batch_tokens = token_tracker.total_tokens - batch_start_tokens
    token_tracker.add_batch_stats(batch_num, len(conversations), batch_tokens)
    
    total_time = time.time() - start_time
    print(f"⭐ 배치 {batch_num} 완료: {len(conversations)}/{target_samples}개 생성, 토큰: {batch_tokens:,}, 소요시간: {int(total_time//60)}분 {int(total_time%60)}초\n")
    
    # DataFrame 생성
    if conversations:
        df = pd.DataFrame(conversations)
        return df
    else:
        return pd.DataFrame()


print("✅ 배치 생성 함수 정의 완료")

✅ 배치 생성 함수 정의 완료


9. 데이터 저장 함수

In [12]:
def save_batch_data(df: pd.DataFrame, batch_num: int):
    """배치 데이터 저장 (CSV, JSON)"""
    
    # 기본 버전 (dialogue, label만)
    df_basic = df[['dialogue', 'label']].copy()
    basic_csv = os.path.join(OUTPUT_DIR, f"s0_batch_{batch_num:02d}_basic.csv")
    df_basic.to_csv(basic_csv, index=False, encoding='utf-8-sig')
    
    # 상세 버전 (모든 컬럼)
    detailed_csv = os.path.join(OUTPUT_DIR, f"s0_batch_{batch_num:02d}_detailed.csv")
    df.to_csv(detailed_csv, index=False, encoding='utf-8-sig')
    
    # JSON 버전
    json_file = os.path.join(OUTPUT_DIR, f"s0_batch_{batch_num:02d}.json")
    df.to_json(json_file, orient='records', force_ascii=False, indent=2)
    
    print(f"💾 배치 {batch_num} 저장 완료:")
    print(f"   - {basic_csv}")
    print(f"   - {detailed_csv}")
    print(f"   - {json_file}\n")


def save_all_data(all_dfs: List[pd.DataFrame]):
    """모든 배치 데이터 통합 저장"""
    
    # 모든 배치 통합
    df_all = pd.concat(all_dfs, ignore_index=True)
    
    # 기본 버전
    df_basic = df_all[['dialogue', 'label']].copy()
    basic_csv = os.path.join(OUTPUT_DIR, "s0_all_data_basic.csv")
    df_basic.to_csv(basic_csv, index=False, encoding='utf-8-sig')
    
    # 상세 버전
    detailed_csv = os.path.join(OUTPUT_DIR, "s0_all_data_detailed.csv")
    df_all.to_csv(detailed_csv, index=False, encoding='utf-8-sig')
    
    # JSON 버전
    json_file = os.path.join(OUTPUT_DIR, "s0_all_data.json")
    df_all.to_json(json_file, orient='records', force_ascii=False, indent=2)
    
    print(f"💾 통합 데이터 저장 완료:")
    print(f"   - {basic_csv}")
    print(f"   - {detailed_csv}")
    print(f"   - {json_file}\n")
    
    return df_all


print("✅ 데이터 저장 함수 정의 완료")

✅ 데이터 저장 함수 정의 완료


10. 메인 생성 함수

In [13]:
def main_generation():
    """메인 생성 프로세스"""
    
    print("\n" + "="*80)
    print("🚀 S0 정상 업무 대화 데이터셋 생성 시작")
    print("="*80)
    print(f"총 시나리오: {len(ALL_SCENARIOS)}개")
    print(f"목표 샘플: {TOTAL_TARGET_SAMPLES}개")
    print(f"배치 수: {NUM_BATCHES}개 (각 {BATCH_SIZE}개)")
    print("="*80 + "\n")
    
    if not client:
        print("❌ 클라이언트가 초기화되지 않았습니다.")
        return
    
    all_dfs = []
    overall_start = time.time()
    
    # 시나리오를 배치별로 분할
    scenarios_per_batch = len(ALL_SCENARIOS) // NUM_BATCHES
    
    for batch_num in range(1, NUM_BATCHES + 1):
        # 이 배치에서 사용할 시나리오 선택
        start_idx = (batch_num - 1) * scenarios_per_batch
        if batch_num == NUM_BATCHES:
            # 마지막 배치는 남은 모든 시나리오 사용
            end_idx = len(ALL_SCENARIOS)
        else:
            end_idx = start_idx + scenarios_per_batch
        
        batch_scenarios = ALL_SCENARIOS[start_idx:end_idx]
        
        # 배치 생성
        df_batch = generate_batch(
            client=client,
            batch_num=batch_num,
            scenarios=batch_scenarios,
            target_samples=BATCH_SIZE,
            delay=API_DELAY_SECONDS
        )
        
        if not df_batch.empty:
            # 배치 데이터 저장
            save_batch_data(df_batch, batch_num)
            all_dfs.append(df_batch)
        
        print("="*80)
        print(f"✅ 배치 {batch_num}/{NUM_BATCHES} 완료")
        print("="*80 + "\n\n")
    
    # 모든 데이터 통합 저장
    if all_dfs:
        df_all = save_all_data(all_dfs)
        
        # 최종 통계
        print("="*80)
        print("📊 최종 통계")
        print("="*80)
        print(f"총 생성 데이터: {len(df_all)}개")
        
        # 대화 턴 수 통계
        df_all['turn_count'] = df_all['dialogue'].apply(lambda x: len(x.split('\n')))
        print(f"\n대화 턴 수:")
        print(f"  평균: {df_all['turn_count'].mean():.1f}")
        print(f"  범위: {df_all['turn_count'].min()} ~ {df_all['turn_count'].max()}")
        
        # 생성된 이름 수
        print(f"\n생성된 고유 이름 수: {name_generator.generation_count}개")
        print(f"사용된 고유 이름 수: {len(name_generator.used_names)}개")
        
        # 토큰 사용량
        token_tracker.print_summary()
        
        # 총 소요 시간
        total_time = time.time() - overall_start
        print(f"\n⏱️  총 소요 시간: {total_time/60:.1f}분 ({total_time:.1f}초)")
        
        print("\n✅ 전체 생성 완료!")
        print("="*80)
    else:
        print("❌ 생성된 데이터가 없습니다.")


print("✅ 메인 생성 함수 정의 완료")

✅ 메인 생성 함수 정의 완료


11. 실행

In [14]:
# 실행
if __name__ == "__main__":
    main_generation()


🚀 S0 정상 업무 대화 데이터셋 생성 시작
총 시나리오: 559개
목표 샘플: 20개
배치 수: 1개 (각 20개)


📦 배치 1 생성 시작 (시나리오 559개 → 목표 20개)

✅ [1/559] 서버 상태 보고: 1개 생성 (민하민, 정하민)
✅ [2/559] DB 백업 협의: 1개 생성 (임세영, 민하준)
✅ [3/559] 프로젝트 진행 상황 공유: 1개 생성 (오서윤, 문준우)
✅ [4/559] 인프라 변경 승인 요청: 1개 생성 (한하율, 윤지호)
✅ [5/559] 보안 이슈 보고: 1개 생성 (허지민, 최수연)
📊 배치 1: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.9% (5/559) [0:37<68:52, 7.46s/it] | 생성: 5개

✅ [6/559] 계정 권한 검토: 1개 생성 (남승호, 전민채)
✅ [7/559] 로그 분석 회의: 1개 생성 (안주호, 임주하)
✅ [8/559] 패치 적용 일정 협의: 1개 생성 (신하성, 노지훈)
✅ [9/559] VPN 접속 기록 점검: 1개 생성 (장윤재, 민서담)
✅ [10/559] 보안 정책 업데이트 공유: 1개 생성 (배하람, 유하영)
📊 배치 1: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 1.8% (10/559) [1:18<71:51, 7.85s/it] | 생성: 10개

✅ [11/559] 데이터 암호화 적용 협의: 1개 생성 (신서유, 장도겸)
✅ [12/559] 외부 감사 대응 회의: 1개 생성 (주예담, 남성민)
✅ [13/559] 보안 알람 설정 검토: 1개 생성 (오하율, 양태린)
✅ [14/559] 서버 접근 권한 요청 검토: 1개 생성 (한건우, 신도율)
✅ [15/559] DB 접근 로그 분석: 1개 생성 (이태윤, 오예솔)
📊 배치 1: [█░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 2.7% (15/559) [2:01<73:33, 8.11s/it] | 생성: 15개
